In [1]:
import numpy as np
import pandas as pd
import os

from get_db_url import get_db_url

As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the obervations from 2017.

# 1

Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

In [2]:
# sql = '''
# SELECT
#     bedroomcnt,
#     bathroomcnt,
#     calculatedfinishedsquarefeet,
#     taxvaluedollarcnt,
#     yearbuilt,
#     taxamount,
#     fips,
#     propertylandusedesc
# FROM properties_2017
# JOIN propertylandusetype
#     ON propertylandusetype.propertylandusetypeid = properties_2017.propertylandusetypeid
#     AND propertylandusetype.propertylandusedesc = 'Single Family Residential';
# '''

# properties = pd.read_sql(sql, get_db_url('zillow'))
# properties.head(2)

properties = pd.read_csv('zillow.csv')

In [3]:
properties.shape

(2152863, 8)

In [4]:
properties.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2152852.0,3.287196,0.954754,0.00,3.00,3.00,4.00,25.00
bathroomcnt,2152852.0,2.230688,0.999280,0.00,2.00,2.00,3.00,32.00
calculatedfinishedsquarefeet,2144379.0,1862.855178,1222.125124,1.00,1257.00,1623.00,2208.00,952576.00
taxvaluedollarcnt,2152370.0,461896.237963,699676.049600,1.00,188170.25,327671.00,534527.00,98428909.00
yearbuilt,2143526.0,1960.949681,22.162196,1801.00,1949.00,1958.00,1976.00,2016.00
taxamount,2148421.0,5634.865978,8178.910249,1.85,2534.98,4108.95,6414.32,1337755.86
fips,2152863.0,6048.377335,20.433292,6037.00,6037.00,6037.00,6059.00,6111.00


In [5]:
properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 8 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
 7   propertylandusedesc           object 
dtypes: float64(7), object(1)
memory usage: 131.4+ MB


In [6]:
properties.isnull().sum()

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
propertylandusedesc                0
dtype: int64

In [7]:
# Let's cache this because that query took a while to run
# properties.to_csv('zillow.csv', index = False)

# 2

Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaninful; remember to document your process and decisions using markdown and code commenting where helpful.

In [8]:
# We have several columns with missing values. Let's see the proportions of these missing values per columns.
properties.isnull().mean()

bedroomcnt                      0.000005
bathroomcnt                     0.000005
calculatedfinishedsquarefeet    0.003941
taxvaluedollarcnt               0.000229
yearbuilt                       0.004337
taxamount                       0.002063
fips                            0.000000
propertylandusedesc             0.000000
dtype: float64

In [9]:
# The missing values make up only a small percentage of the whole in each column. Let's how many rows have missing values.
(properties.isnull().sum(axis = 1) != 0).sum(), (properties.isnull().sum(axis = 1) != 0).mean()

(12628, 0.005865677472277613)

Roughly half a percent of the rows comprises of missing values. Let's try to get a little more insight here.

In [10]:
# How many rows have 2 or more missing values?
(properties.isnull().sum(axis = 1) >= 2).sum(), (properties.isnull().sum(axis = 1) >= 2).mean()

(8043, 0.0037359553301812517)

In [11]:
# What's our distribution of missing values when we look at rows with 1 missing value or less?
rows_with_one_or_less_missing_values = properties.isnull().sum(axis = 1) < 2
properties[rows_with_one_or_less_missing_values].isnull().sum()

bedroomcnt                         0
bathroomcnt                        0
calculatedfinishedsquarefeet     560
taxvaluedollarcnt                 18
yearbuilt                       1439
taxamount                       2568
fips                               0
propertylandusedesc                0
dtype: int64

We will need square footage for the first iteration of our model so we will remove all rows missing that value as well as any rows missing more than one value. We'll also remove rows missing taxvaluedollarcnt since this is our target variable. We will impute everything else.

In [12]:
# Remove rows with two or more missing values.
properties = properties[rows_with_one_or_less_missing_values]

# Remove rows missing square footage
rows_not_missing_square_feet = properties.calculatedfinishedsquarefeet.notnull()
properties = properties[rows_not_missing_square_feet]

# Remove rows missing taxvaluedollarcnt
rows_not_missing_taxvalue = properties.taxvaluedollarcnt.notnull()
properties = properties[rows_not_missing_taxvalue]

In [13]:
properties.isnull().sum()

bedroomcnt                         0
bathroomcnt                        0
calculatedfinishedsquarefeet       0
taxvaluedollarcnt                  0
yearbuilt                       1439
taxamount                       2568
fips                               0
propertylandusedesc                0
dtype: int64

In [14]:
# We'll fill in yearbuilt with the mode.
properties.yearbuilt.mode()[0]

1955.0

In [15]:
properties.yearbuilt.fillna(properties.yearbuilt.mode()[0], inplace = True)

# Let's also cast the column to int.
properties.yearbuilt = properties.yearbuilt.astype('int')

In [16]:
properties.isnull().sum()

bedroomcnt                         0
bathroomcnt                        0
calculatedfinishedsquarefeet       0
taxvaluedollarcnt                  0
yearbuilt                          0
taxamount                       2568
fips                               0
propertylandusedesc                0
dtype: int64

In [17]:
properties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2144242 entries, 4 to 2152862
Data columns (total 8 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     int64  
 5   taxamount                     float64
 6   fips                          float64
 7   propertylandusedesc           object 
dtypes: float64(6), int64(1), object(1)
memory usage: 147.2+ MB


In [18]:
# Finally let's fill in taxamount with the mean.
properties.taxamount.mean()

5616.12711389353

In [19]:
properties.taxamount.fillna(properties.taxamount.mean(), inplace = True)

In [20]:
properties.isnull().sum()

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
yearbuilt                       0
taxamount                       0
fips                            0
propertylandusedesc             0
dtype: int64

In [21]:
properties.shape

(2144242, 8)

In [22]:
# There was also some weird values in the square footage column where the value was 1. Let's take a look.

properties.calculatedfinishedsquarefeet.value_counts().sort_index().head(30)

1.0      6
2.0      2
3.0      7
7.0      1
10.0     2
12.0     1
20.0    29
24.0     1
29.0     1
30.0     3
31.0     1
32.0     1
40.0    13
43.0     1
48.0     3
54.0     1
55.0     1
56.0     1
60.0    25
63.0     1
64.0     1
65.0     1
66.0     1
72.0     3
76.0     1
77.0     1
78.0     1
80.0     1
90.0     1
95.0     1
Name: calculatedfinishedsquarefeet, dtype: int64

Based on some quick research, some of the smallest homes in the world appear to be around 70 square feet. In case some of these entries are legitimate we'll keep everything at 60 square feet or more.

In [23]:
sixty_square_feet_or_more = properties.calculatedfinishedsquarefeet >= 60
properties = properties[sixty_square_feet_or_more]

In [24]:
properties.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2144167.0,3.300090,0.934225,0.00,3.000,3.00,4.000,25.00
bathroomcnt,2144167.0,2.239428,0.991288,0.00,2.000,2.00,3.000,32.00
calculatedfinishedsquarefeet,2144167.0,1862.939873,1222.072751,60.00,1257.000,1623.00,2208.000,952576.00
taxvaluedollarcnt,2144167.0,460605.051426,678463.333888,1.00,188822.000,328000.00,534461.000,90188462.00
yearbuilt,2144167.0,1960.945829,22.153397,1801.00,1949.000,1958.00,1976.000,2016.00
taxamount,2144167.0,5616.251674,7809.257606,6.34,2542.235,4115.18,6407.335,1078101.87
fips,2144167.0,6048.316582,20.363256,6037.00,6037.000,6037.00,6059.000,6111.00


In [25]:
# Let's see the tax values
properties.taxvaluedollarcnt.value_counts().sort_index().head(30)

1.0       1
4.0       1
9.0      15
10.0     22
11.0      1
16.0      1
17.0      1
18.0     67
20.0      3
22.0      8
23.0      6
31.0      1
46.0      1
101.0     1
102.0     1
103.0     4
104.0     6
105.0     1
106.0     3
107.0     2
200.0     1
284.0     1
340.0     1
387.0     1
549.0     1
568.0     2
722.0     1
762.0     1
834.0     1
895.0     1
Name: taxvaluedollarcnt, dtype: int64

We won't do anything more.

In [26]:
# One last thing let's rename these columns

properties = properties.rename(columns = {
    'bedroomcnt' : 'bedroom_count',
    'bathroomcnt' : 'bathroom_count',
    'calculatedfinishedsquarefeet' : 'square_feet',
    'taxvaluedollarcnt' : 'tax_value_dollar_count',
    'yearbuilt' : 'year_built',
    'taxamount' : 'tax_amount',
    'fips' : 'fed_code',
    'propertylandusedesc' : 'property_land_use_description'
})
properties.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2144167 entries, 4 to 2152862
Data columns (total 8 columns):
 #   Column                         Dtype  
---  ------                         -----  
 0   bedroom_count                  float64
 1   bathroom_count                 float64
 2   square_feet                    float64
 3   tax_value_dollar_count         float64
 4   year_built                     int64  
 5   tax_amount                     float64
 6   fed_code                       float64
 7   property_land_use_description  object 
dtypes: float64(6), int64(1), object(1)
memory usage: 147.2+ MB


In [27]:
properties.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroom_count,2144167.0,3.300090,0.934225,0.00,3.000,3.00,4.000,25.00
bathroom_count,2144167.0,2.239428,0.991288,0.00,2.000,2.00,3.000,32.00
square_feet,2144167.0,1862.939873,1222.072751,60.00,1257.000,1623.00,2208.000,952576.00
tax_value_dollar_count,2144167.0,460605.051426,678463.333888,1.00,188822.000,328000.00,534461.000,90188462.00
year_built,2144167.0,1960.945829,22.153397,1801.00,1949.000,1958.00,1976.000,2016.00
tax_amount,2144167.0,5616.251674,7809.257606,6.34,2542.235,4115.18,6407.335,1078101.87
fed_code,2144167.0,6048.316582,20.363256,6037.00,6037.000,6037.00,6059.000,6111.00


All done!

# 3

Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe witn no missing values in your wrangle.py file. Name your final function wrangle_zillow.

In [28]:
# These are the acquisition functions

def get_zillow_data(use_cache: bool = True) -> pd.core.frame.DataFrame:
    '''
        Return a dataframe containing data from the zillow properties dataset.

        If a zillow.csv file containing the data does not already
        exist the data will be cached in that file inside the current
        working directory. Otherwise, the data will be read from the
        .csv file.

        Parameters
        ----------
        use_cache: bool, default True
            If True the dataset will be retrieved from a csv file if one
            exists, otherwise, it will be retrieved from the MySQL database. 
            If False the dataset will be retrieved from the MySQL database
            even if the csv file exists.

        Returns
        -------
        DataFrame: A Pandas DataFrame containing the data from the zillow
            dataset is returned.
    '''

    # If the file is cached, read from the .csv file
    if os.path.exists('zillow.csv') and use_cache:
        return pd.read_csv('zillow.csv')
    
    # Otherwise read from the mysql database
    else:
        df = pd.read_sql(_get_zillow_sql(), get_db_url('zillow'))
        df.to_csv('zillow.csv', index = False)
        return df
    
def _get_zillow_sql() -> str:
    '''
        Returns the SQL code required to retrieve the zillow dataset
        from the MySQL database.
    '''

    return '''
        SELECT
            bedroomcnt,
            bathroomcnt,
            calculatedfinishedsquarefeet,
            taxvaluedollarcnt,
            yearbuilt,
            taxamount,
            fips,
            propertylandusedesc
        FROM properties_2017
        JOIN propertylandusetype
            ON propertylandusetype.propertylandusetypeid = properties_2017.propertylandusetypeid
            AND propertylandusetype.propertylandusedesc = 'Single Family Residential';
    '''

In [33]:
# This is the prepare function

def prepare_zillow_data(df: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    '''
        Returns a prepared zillow dataset with all missing values handled.
        
        Parameters
        ----------
        df: DataFrame
            A pandas dataframe containing the unprepared zillow dataset.
        
        Returns
        -------
        DataFrame: A pandas dataframe containing the prepared zillow dataset.
    '''
    
    # Remove rows with two or more missing values.
    rows_with_one_or_less_missing_values = df.isnull().sum(axis = 1) < 2
    df = df[rows_with_one_or_less_missing_values]

    # Remove rows missing square footage
    rows_not_missing_square_feet = df.calculatedfinishedsquarefeet.notnull()
    df = df[rows_not_missing_square_feet]

    # Remove rows missing taxvaluedollarcnt
    rows_not_missing_taxvalue = df.taxvaluedollarcnt.notnull()
    df = df[rows_not_missing_taxvalue]
    
    # Fill in yearbuilt column with the mode and cast to int
    df.yearbuilt.fillna(df.yearbuilt.mode()[0], inplace = True)
    df.yearbuilt = df.yearbuilt.astype('int')
    
    # Fill in the taxamount column with the mean
    df.taxamount.fillna(df.taxamount.mean(), inplace = True)
    
    # Remove properties with less than 60 square feet
    sixty_square_feet_or_more = df.calculatedfinishedsquarefeet >= 60
    df = df[sixty_square_feet_or_more]

    # Rename the columns for readability
    df = df.rename(columns = {
        'bedroomcnt' : 'bedroom_count',
        'bathroomcnt' : 'bathroom_count',
        'calculatedfinishedsquarefeet' : 'square_feet',
        'taxvaluedollarcnt' : 'tax_value_dollar_count',
        'yearbuilt' : 'year_built',
        'taxamount' : 'tax_amount',
        'fips' : 'fed_code',
        'propertylandusedesc' : 'property_land_use_description'
    })
    
    return df

In [34]:
# Now let's bring it all together

def wrangle_zillow() -> pd.core.frame.DataFrame:
    '''
        Returns the acquired and prepared zillow dataset.
        
        Returns
        -------
        DataFrame: A pandas dataframe containing the prepared zillow dataset.
    '''
    
    return prepare_zillow_data(get_zillow_data())

In [35]:
# Let's test it
df = wrangle_zillow()

In [36]:
df.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2144167 entries, 4 to 2152862
Data columns (total 8 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   bedroom_count                  2144167 non-null  float64
 1   bathroom_count                 2144167 non-null  float64
 2   square_feet                    2144167 non-null  float64
 3   tax_value_dollar_count         2144167 non-null  float64
 4   year_built                     2144167 non-null  int64  
 5   tax_amount                     2144167 non-null  float64
 6   fed_code                       2144167 non-null  float64
 7   property_land_use_description  2144167 non-null  object 
dtypes: float64(6), int64(1), object(1)
memory usage: 147.2+ MB
